In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_modelling.py to data_modelling.py
Saving preprocessor2.py to preprocessor2.py


In [3]:
from preprocessor2 import Preprocessor2
import pandas as pd
from sklearn.metrics import accuracy_score
from data_modelling import DataModelling
from sklearn.model_selection import GridSearchCV

In [6]:
file_path = "/content/drive/MyDrive/Applied Machine Learning/data/steam_data_with_correct_prices.csv"
preprocessor = Preprocessor2(file_path)
game_types_count_value = 5000      #3500
supported_tags_count_value = 5000  #4000
common_languages_count_value = 5000 #3500
drop_reviews = True
cleaned_df = preprocessor.run_preprocessor(game_types_count_value,supported_tags_count_value,common_languages_count_value,drop_reviews)

seasons sorunsuz bir şekilde eklendi.


In [7]:
df = cleaned_df
number_of_target = 9 # it should be 3,5 or 9
apply_data_augmentation_bool = True
data_modelling = DataModelling(df,number_of_target,apply_data_augmentation_bool)

In [8]:
model_df = data_modelling.run_modelling()

In [9]:
# sample = cleaned_df.sample(frac=1)
X = model_df.drop("review_text", axis = 1)
y = model_df["review_text"]
from sklearn.model_selection import train_test_split

# Veriyi train ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from catboost import CatBoostClassifier

# Modeli tanımlama ve eğitme
model = CatBoostClassifier(verbose=False)
model.fit(X_train, y_train)

# Özellik önem skorlarını elde etme
importances = model.get_feature_importance(type="FeatureImportance")

# Test verileri üzerinde tahmin yapma
y_pred = model.predict(X_test)

# Doğruluk hesaplama
accuracy = accuracy_score(y_test, y_pred)
print("Doğruluk (Accuracy): {:.2f}%".format(accuracy * 100))

Doğruluk (Accuracy): 70.22%


In [11]:
# Gerekli kütüphaneleri yükleyin
from catboost import CatBoostClassifier

model = CatBoostClassifier(verbose=0)

# Hiperparametre grid'i tanımlayın
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'l2_leaf_reg': [1, 3, 5]
}

# GridSearchCV'yi tanımlayın
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

# GridSearchCV'yi eğitim verileriyle çalıştırın
grid_search.fit(X_train, y_train)

# En iyi parametreler ve en iyi skoru alın
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("En iyi hiperparametreler:", best_params)
print("En iyi doğruluk skoru:", best_score)

# En iyi modeli test verisi üzerinde değerlendirin
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test seti doğruluk skoru:", test_score)


En iyi hiperparametreler: {'depth': 8, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.2}
En iyi doğruluk skoru: 0.7112669600293362
Test seti doğruluk skoru: 0.7148270382593815
